In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import csv
import pandas as pd
import numpy as np
import tflearn

In [ ]:
def get_dataset(name):
    df = pd.read_csv(name, sep=';', index_col=0, encoding="Latin1")
    df = df[::-1]
    df.columns = ['Siste', 'Kjoper', 'Selger', 'Hoy', 'Lav', 'Totalt_omsatt_NOK', 'Totalt_antall_aksjer_omsatt', 'Antall_off_handler', 'Antall_handler_totalt', 'VWAP']
    return df


def get_currency_data(name):
    df = pd.read_csv(name, sep=';', index_col=0)
    return df


def append_currency_to_stock_price_df(df_stock, df_currency):
    df_stock["Dollarkurs"] = df_currency["1 USD"]


MHG = get_dataset("data\\MHG_data.csv")
nok = get_currency_data("data\\Valutakurser.csv")
nok.fillna(-999999, inplace=True)
print nok.head()

append_currency_to_stock_price_df(MHG, nok)
print MHG


In [ ]:
def plot_dataframe(df):
    df.plot()
    plt.show()
    

def plot_siste(df):
    df = df.Siste
    df.plot()
    plt.show()

    
def plot_kjoper(df):
    df = df.Kjoper
    df.plot()
    plt.show()
    
    
def plot_omsatt_nok(df):
    df = df.Totalt_omsatt_NOK
    df.plot()
    plt.show()
    
    
def plot_omsatt_antall_aksjer(df):
    df = df.Totalt_antall_aksjer_omsatt
    df.plot()
    plt.show()

def plot_antall_handler_off(df):
    df = df.Antall_off_handler
    df.plot()
    plt.show()
    
    
def plot_antall_handler_totalt(df):
    df = df.Antall_handler_totalt
    df.plot()
    plt.show()


def plot_VWAP(df):
    df = df.VWAP
    df.plot()
    plt.show()

In [ ]:
plot_siste(MHG)
plot_kjoper(MHG)
plot_omsatt_nok(MHG)
plot_omsatt_antall_aksjer(MHG)
plot_antall_handler_off(MHG)
plot_antall_handler_totalt(MHG)
plot_VWAP(MHG)

In [ ]:

# Setup neural network

days_history = 10

import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)

# Build deep neural network
input_layer = tflearn.input_data(shape=[None, days_history])

dense1 = tflearn.fully_connected(input_layer, 10, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)

dropout1 = tflearn.dropout(dense1, 0.8)

softmax = tflearn.fully_connected(dropout1, 2, activation='softmax')

# Regression using SGD with learning rate decay and Top-1 accuracy
sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.96, decay_step=1000)
top_k = tflearn.metrics.Top_k(1)
net = tflearn.regression(softmax, optimizer=sgd, metric=top_k,
                         loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X, Y, n_epoch=20, validation_set=(testX, testY),
          show_metric=True, run_id="dense_model")